# Load packages

Read in all necessary packages:

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas.io.parsers import read_csv
import scvelo as scv
import scanpy as sc
import numpy as np
from functools import reduce
from anndata import AnnData, read_h5ad
import singlecellmultiomics.bamProcessing.bamToRNACounts
import loompy

In [ ]:
scv.settings.verbosity = 3 # show errors(0), warnings(1), info(2), hints(3)
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

scv.settings.presenter_view = True  # set max width size for presenter view
#scv.set_figure_params('scvelo')  # for beautified visualization
sc.set_figure_params(dpi=100, color_map = 'viridis')

# Load the dataset

In [ ]:
pathToData = '/Users/m.blotenburg/Documents/Projects/TCHIC/data/rep3_rep4/'

In [ ]:
adata = sc.read(pathToData + 'dataframes/20210615_all_OUD5651_OUD5771_OUD5772_OUD6104_OUD5886_rep234_day34567_Scanpy.h5ad')

In [ ]:
barcodes = pd.read_csv("/Users/m.blotenburg/Documents/Projects/Helena/data/VASAbarcodes.csv", sep = '\t', header = None, index_col=0, squeeze=True).to_dict()
barcodes = {y:x for x,y in barcodes.items()}
adata.obs['cellnames'] = ['-'.join(x) for x in [ob.split('-')[0:7] for ob in adata.obs.index]]
adata.obs['bc'] = [ob.split(':')[1] for ob in adata.obs['cellnames']]
adata.obs['bc'] = adata.obs['bc'].map(lambda x: barcodes[x])
adata.obs['cellname'] = adata.obs['batch'].astype(str) + '_' + adata.obs['bc'].astype(str)
adata.obs.index = adata.obs['cellname']
adata.obs.index.rename('index')

adata.obs.head(2)

In [ ]:
adata.obs.index = adata.obs.index.rename('index')
adata.obs.head(2)

In [ ]:
cells_clusters = pd.DataFrame(adata.obs['leiden'])
cells_clusters.head(2)
#cells_clusters.to_csv(pathToData + 'clusters.csv')

In [ ]:
pathToData

In [ ]:
cells_clusters.to_csv(pathToData + 'dataframes/20210623_rep234_day34567_cells_clusters_leiden.csv', sep = '\t')

In [ ]:
sc.get.obs_df(
        adata,
        obsm_keys=[("X_umap", 0), ("X_umap", 1)]).to_csv(pathToData + 'dataframes/20210623_rep234_day34567_cells_umapCoordinates.csv', sep = '\t')

# RNA velocity

In [ ]:
scv.tl.velocity(adata)

In [ ]:
scv.tl.velocity_graph(adata)

In [ ]:
scv.pl.proportions(adata, groupby='leiden_general')

## Project velocities

We can choose different ways of visualising the velocity onto the umap

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
scv.pl.velocity_embedding_stream(adata, ax=ax, basis='umap', color = 'day', alpha=0.8)#, palette=sns.color_palette("hls", 17))

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
scv.pl.velocity_embedding(adata, ax=ax, arrow_length=3, arrow_size=2, dpi=120, color = 'day', alpha=0.8)#, palette=sns.color_palette("hls", 17))

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
scv.pl.velocity_embedding_grid(adata, ax=ax, basis='umap', color = 'day', alpha=0.8)#, palette=sns.color_palette("hls", 17))

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
scv.pl.velocity_graph(adata, ax=ax, color = 'day',alpha=0.8)#, palette=sns.color_palette("hls", 17))

The cell cycle detected by RNA velocity, is biologically affirmed by cell cycle scores (standardized scores of mean expression levels of phase marker genes).

In [ ]:
scv.tl.score_genes_cell_cycle(adata)
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.scatter(adata, ax=ax,color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95])

Two more useful stats: - The speed or rate of differentiation is given by the length of the velocity vector. - The coherence of the vector field (i.e., how a velocity vector correlates with its neighboring velocities) provides a measure of confidence.

In [ ]:
scv.tl.velocity_confidence(adata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata, c=keys, cmap='coolwarm', perc=[5, 95])

In [ ]:
df = adata.obs.groupby('celltype')[keys].mean().T
df.style.background_gradient(cmap='coolwarm', axis=1)

We can also calculate and plot the velocity pseudotime

In [ ]:
scv.tl.velocity_pseudotime(adata, n_dcs=50)
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.scatter(adata, ax=ax, color='velocity_pseudotime', cmap='gnuplot')

In [ ]:
scv.tl.velocity_pseudotime(adata, groupby='day',groups='Gastd7')
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.scatter(adata, ax=ax, color='velocity_pseudotime', cmap='gnuplot')

In [ ]:
scv.tl.velocity_pseudotime(adata, groupby='leiden_general')
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.scatter(adata, ax=ax, color='velocity_pseudotime', cmap='gnuplot')

Trace individual cells

In [ ]:
x = adata.obs[adata.obs['day'] == 'Gastd5']
x[x['leiden_general'] == 'Neural'].tail(5)

In [ ]:
adata.obs.index.get_indexer_for((adata.obs[adata.obs.index == 'PZ-MB-TChIC-Gastd5-rep4-H3K27me3-6:TGAAAGGC'].index))

In [ ]:
x, y = scv.utils.get_cell_transitions(adata, basis='umap', starting_cell=21478)
ax = scv.pl.velocity_graph(adata, c='lightgrey', edge_width=.05, show=False)
ax = scv.pl.scatter(adata, x=x, y=y, s=120, c='ascending', cmap='gnuplot', ax=ax)


To see the different batches on top of the 'velocity' umap, just change the 'color'.

In [ ]:
#scv.settings.set_figure_params('scvelo')
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.velocity_embedding_stream(adata, ax=ax,
                                 basis='umap', 
                                 color=['day'])#,palette=sns.color_palette("hls", 13))

In [ ]:
#scv.settings.set_figure_params('scvelo')
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.velocity_embedding_stream(adata, ax=ax,
                                 basis='umap', 
                                 color='batch')#,palette=sc.pl.palettes.default_20, size = 60)

In [ ]:
#scv.settings.set_figure_params('scvelo')
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.velocity_embedding_stream(adata, 
                                 basis='umap', ax=ax,
                                 color='mark')

Plotting our favourite genes in terms of velocity:

In [ ]:
sc.pl.umap(adata, color = "T", color_map = "viridis", use_raw=True, frameon=False, size = 80)

In [ ]:
sc.pl.umap(adata, color = "T", color_map = "viridis", layer = 'spliced', frameon=False, size = 80)

In [ ]:
sc.pl.umap(adata, color = "T", color_map = "viridis", layer = 'unspliced', frameon=False, size = 80)

In [ ]:
scv.pl.scatter(adata, ['Pou5f1', 'Onecut2', 'T','Sox17'], color=['day'], size = 50, figsize = (20,20))



In [ ]:
scv.pl.velocity(adata, var_names='T', color = "day", 
               # palette=sns.color_palette('tab20',13), 
                ncols=1) + sc.pl.umap(adata, color='day', #palette=sns.color_palette('tab20',13), 
                                                                             legend_loc = "on data", title='Cell types', legend_fontsize=10, frameon=False, size = 40)

In [ ]:
polycomb = ['Jarid2', "Eed","Ezh2", "Prc1", "Ring1", "Suz12", "Phc1","Phc2", "Suv39h2"] #Prc2
k4 = ['Setd5', 'Setd1a', 'Setd3', 'Setdb2', 'Setd7', 'Setd1b','Jarid2']

markergenes = ['Esrrb', 'Dppa2', 'Dppa4', 'Nanog', 'Dppa5a','Gsc', 'Klf9', 'Klf2', 'Utf1', 'Pim2', 
               'Dppa3', 'Dnmt3b', 'Dazl', #PGC
               'Cdh1', 'Trh', 'Sox17', 'Nodal', 'Epcam', 'Foxa2', 'Lefty1', #endoderm/PGC/node
               'Sox7','Kdr','Car2', #haemogenic
               'Bmp4', 'Gata4', 'Gata6', 'Hand1', 'Hand2', #heart
               'Dll1', 'Hes7', 'Uncx', 'Snai1', 'Hes5', 'Dkk1', #presomitic
               'Eya1', 'Pax3', 'Six1', 'Cer1', #early somite
               'Aldh1a2', 'Notch1', 'Lfng', #wavefront
               'Onecut2', 'Elavl3', 'Crabp1', #neural tube/branchial arches
               'Wnt3a', 'Fgf8', 'T', 'Cdx2', #posterior
               'Sox21', 'Nkx1-2', #neural SOX2 SOX1
               'Prdm1', 'Irx1', 'Eomes', 'Meox1','Tbx4', 'Sox3','H19', 'Xist','Gata2', 'Gata3','Gata4' #other   
              ]

hox = adata.var.filter(regex=r'Hox', axis=0).index

In [ ]:
for gene in polycomb:
    scv.pl.scatter(adata, gene, color=['day'], size = 50, figsize = (4,4), alpha=0.7)


In [ ]:
for gene in hox:
    scv.pl.scatter(adata, gene, color=['day'], size = 50, figsize = (4,4), alpha=0.7)




In [ ]:
for gene in k4:
       scv.pl.scatter(adata, gene, color=['day'], size = 50, figsize = (4,4), alpha=0.7)




In [ ]:
for gene in markergenes:
       scv.pl.scatter(adata, gene, color=['day'], size = 50, figsize = (4,4), alpha=0.7)





In [ ]:
scv.pl.velocity(adata, var_names=markergenes, color = "leiden_general", ncols=1, size = 30, alpha=0.5)

In [ ]:
scv.pl.velocity(adata, var_names=markergenes, color = "day", ncols=1)

To define the top velocity genes, we need to rank them per cluster.

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
scv.tl.rank_velocity_genes(adata, groupby = 'leiden', min_counts = 1, resolution=0.4,min_corr=.3)

df_velo = pd.DataFrame(adata.uns['rank_velocity_genes']['names'])
df_velo

In [ ]:
scv.tl.rank_velocity_genes(adata, groupby = 'day', min_counts = 1, resolution=0.4,min_corr=.3)

df_day_velo = pd.DataFrame(adata.uns['rank_velocity_genes']['names'])
df_day_velo

We can then visualise these too.

In [ ]:
scv.pl.scatter(adata, df_day_velo['Gastd4'][:5], ylabel='Gastd4')


In [ ]:
scv.pl.velocity(adata, var_names=pd.DataFrame(adata.uns['rank_velocity_genes']['names']).head(2).values.flatten(), ncols=1, colorbar=True)

# PAGA graph building

In [ ]:
# this is needed due to a current bug - bugfix is coming soon.
adata.uns['neighbors']['distances'] = adata.obsp['distances']
adata.uns['neighbors']['connectivities'] = adata.obsp['connectivities']

scv.tl.paga(adata, groups='leiden')
df = scv.get_df(adata, 'paga/transitions_confidence', precision=2).T
df.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
scv.pl.paga(adata, basis='umap', size=50, alpha=.1,ax=ax,color = 'leiden',
            min_edge_width=2, node_size_scale=1.5)

In [ ]:
pd.DataFrame(adata.obs['leiden_general']).to_csv(pathToData + '20210622_rep234_day34567_cellsLeidenClusters_general.csv')

# Save

During the course of this analysis, the AnnData accumlated the following annotations.

In [ ]:
adata

In [ ]:
adata.write(results_file, compression='gzip')  # `compression='gzip'` saves disk space, but slows down writing and subsequent reading

Get a rough overview of the file using `h5ls`, which has many options - for more details see [here](https://github.com/theislab/scanpy_usage/blob/master/170505_seurat/info_h5ad.md). The file format might still be subject to further optimization in the future. All reading functions will remain backwards-compatible, though.

If you want to share this file with people who merely want to use it for visualization, a simple way to reduce the file size is by removing the dense scaled and corrected data matrix. The file still contains the raw data used in the visualizations.

In [ ]:
adata.X = None
adata.write('./write/Scanpy120hAA.h5ad', compression='gzip')

If you want to export to "csv", you have the following options:

In [ ]:
# Export single fields of the annotation of observations
# adata.obs[['n_counts', 'louvain_groups']].to_csv(
#     './write/pbmc3k_corrected_louvain_groups.csv')

# Export single columns of the multidimensional annotation
# adata.obsm.to_df()[['X_pca1', 'X_pca2']].to_csv(
#     './write/pbmc3k_corrected_X_pca.csv')

# Or export everything except the data using `.write_csvs`.
# Set `skip_data=False` if you also want to export the data.
# adata.write_csvs(results_file[:-5], )